In [ ]:
import os
from utils import *
import optuna

In [ ]:
data, usermap, itemmap, users = load_data2()
data_train, data_test, data_val = split_data2(data, 0, 0.2)

In [ ]:
study_name = "Easer"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

def objective(trial):
    topK = trial.suggest_int('topK', 0, 1000)
    l2_norm = trial.suggest_float('l2_norm', 1, 1e10, log=True)
    
    recommender = EASE_R_Recommender(data_train, verbose=False)
    recommender.fit(
        topK=topK if topK > 0 else None,
        l2_norm=l2_norm,
        normalize_matrix=False,
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=150)

In [ ]:
study.best_params

In [ ]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender = EASE_R_Recommender(data_train, verbose=False)
recommender.fit(**study.best_params)

In [ ]:
pr, re, ev_map, ev_ndcg, ev_auc = evaluator(recommender, data_train, data_val)
print("Precision: {:.8f}".format(pr))
print("Recall: {:.8f}".format(re))
print("MAP: {:.8f}".format(ev_map))

In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)